<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/06_combined_model_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [3]:
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

!pip install -q tensorflow-io
import tensorflow_io as tfio

from utils import get_train_val_ds, get_class_weight

     |████████████████████████████████| 25.9 MB 1.5 MB/s 


In [4]:
BATCH_SIZE = 32
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b3/feature_vector/2'

In [5]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [6]:
train_ds, val_ds = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 16580 files belonging to 16 classes.
Found 3495 files belonging to 16 classes.


In [7]:
classes, class_weight = get_class_weight()

In [8]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

val_ds = (
    val_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [17]:
rgb_input = keras.Input(shape=IMG_SIZE+(3,))
rgb_hub_module = hub.KerasLayer(HUB_URL,  trainable=True)
hsv_hub_module = hub.KerasLayer(HUB_URL,  trainable=True)
xyz_hub_module = hub.KerasLayer(HUB_URL,  trainable=True)

rgb_features = rgb_hub_module(rgb_input)

xyz_input = tfio.experimental.color.rgb_to_xyz(rgb_input)
xyz_features = xyz_hub_module(xyz_input)

hsv_input = tf.image.rgb_to_hsv(rgb_input)
hsv_features = hsv_hub_module(hsv_input)

features = keras.layers.Concatenate()([rgb_features, xyz_features, hsv_features])

output = keras.layers.Dense(N_CLASSES, activation='softmax')(features)

model = keras.models.Model(inputs=[rgb_input], outputs=[output])

In [18]:
model._name = 'combined-model'

In [20]:
metrics = [
            keras.metrics.AUC(name='auc', curve='PR', num_thresholds=100),
            'accuracy'
          ]

loss_fn = keras.losses.CategoricalCrossentropy()
lr_schedule =tf.keras.optimizers.schedules.ExponentialDecay(1e-3, 500, .9)

model.compile(loss=loss_fn,
            optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
            metrics=metrics)

In [21]:
early_stopping = keras.callbacks.EarlyStopping(
                                              monitor='val_auc', 
                                              verbose=1,
                                              patience=5,
                                              mode='max')

check_point_path = os.path.join('./logs/checkpoints', model._name+".h5")
check_point = keras.callbacks.ModelCheckpoint(
                                            filepath=check_point_path,
                                            monitor='val_auc',
                                            save_best_only=True,
                                            mode='max')
        
callbacks = [early_stopping, check_point]

In [22]:
history = model.fit(train_ds,
                    callbacks=callbacks,
                    epochs=15,
                    validation_data=val_ds,
                    class_weight=class_weight)

Epoch 1/15


ResourceExhaustedError: ignored

In [ ]:
!cp ./logs/checkpoints/combined-model-10.h5 ./drive/MyDrive/checkpoints/combined-model-10.2.h5
!cp ./logs/checkpoints/combined-model-15.h5 ./drive/MyDrive/checkpoints/combined-model-15.2.h5
!cp ./logs/checkpoints/combined-model-20.h5 ./drive/MyDrive/checkpoints/combined-model-20.2.h5

In [ ]:
history = model.fit(train_ds,
                    callbacks=callbacks,
                    epochs=30,
                    initial_epoch=20,
                    class_weight=class_weight)

Epoch 21/30
533/533 [==============================] - 3123s 6s/step - loss: 5.6207 - auc: 0.9592 - accuracy: 0.9698
Epoch 22/30
533/533 [==============================] - 449s 839ms/step - loss: 4.0567 - auc: 0.9673 - accuracy: 0.9739
Epoch 23/30
533/533 [==============================] - 448s 839ms/step - loss: 3.2036 - auc: 0.9746 - accuracy: 0.9780
Epoch 24/30
533/533 [==============================] - 448s 838ms/step - loss: 3.1084 - auc: 0.9736 - accuracy: 0.9782
Epoch 25/30
533/533 [==============================] - 448s 839ms/step - loss: 2.2643 - auc: 0.9796 - accuracy: 0.9810
Epoch 26/30
533/533 [==============================] - 448s 839ms/step - loss: 2.1500 - auc: 0.9825 - accuracy: 0.9830
Epoch 27/30
533/533 [==============================] - 449s 840ms/step - loss: 1.4366 - auc: 0.9879 - accuracy: 0.9886
Epoch 28/30
533/533 [==============================] - 448s 838ms/step - loss: 1.4828 - auc: 0.9863 - accuracy: 0.9868
Epoch 29/30
533/533 [=============================

In [ ]:
!cp ./logs/checkpoints/combined-model-25.h5 ./drive/MyDrive/checkpoints/combined-model-25.2.h5
!cp ./logs/checkpoints/combined-model-30.h5 ./drive/MyDrive/checkpoints/combined-model-30.2.h5